# 异步IO-网络IO和文件IO实例

In [1]:
# encoding: utf8

import os
import re
import time
import asyncio
import aiohttp
import aiofiles

# search all pdf file names.
pdf_pattern = re.compile(">(\d+.+\.pdf)<")
PROJECT_DIR = os.getcwd()

home_url = 'https://courses.cs.washington.edu/courses/cse326/08sp/lectures/markup/'


async def download(url: str):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            content = await resp.read()
            return content


async def persistent(content: bytes, file_path: str):
    async with aiofiles.open(file_path, 'wb+') as f:
        await f.write(content)


async def download_and_persistent(url: str, file_path: str):
    content = await download(url)
    await persistent(content, file_path)

    print(f'download {os.path.basename(file_path)} size: {len(content) // 1024} KB done.')


async def main():
    print(f'Get {home_url} pdfs.')
    content = await download(home_url)
    content = content.decode()

    tasks = []
    for pdf in pdf_pattern.findall(content):
        url = os.path.join(home_url, pdf)
        print(f'download {pdf}.')
        file_path = os.path.join(PROJECT_DIR, pdf)

        task = asyncio.create_task(download_and_persistent(url, file_path))
        tasks.append(task)

    await asyncio.gather(*tasks)

In [2]:
start_time = time.time()
print(f'Start at {time.strftime("%X")}')
await main()
finish_time = time.time()
print(f'Finished at {time.strftime("%X")}, elapse {int(finish_time - start_time)} seconds')

Start at 20:18:05
Get https://courses.cs.washington.edu/courses/cse326/08sp/lectures/markup/ pdfs.
download 01-intro-markup.pdf.
download 02-asymptotics-markup.pdf.
download 03-binary-heaps-markup.pdf.
download 04-d-heaps-markup.pdf.
download 05-leftist-heaps-markup.pdf.
download 06-skew-heaps-markup.pdf.
download 07-binomial-queues-markup.pdf.
download 08-bst-markup.pdf.
download 09-AVL-deletion-markup.pdf.
download 09-avl-markup.pdf.
download 10-splay-markup.pdf.
download 11-b-trees-markup.pdf.
download 12-other-balanced-bsts-markup.pdf.
download 13-hashing-markup.pdf.
download 14-sorting-markup.pdf.
download 15-mergesort-markup.pdf.
download 16-quicksort-compare-sort-markup.pdf.
download 17-bucket-and-radix-sort-markup.pdf.
download 18-disjoint-union-find-markup.pdf.
download 19-disjoint-union-find-part-2-markup.pdf.
download 20-graphs-markup.pdf.
download 21-shortest-paths-markup.pdf.
download 22-spanning-trees-markup.pdf.
download 22-spanning-trees.pdf.
download 23-NP-completeness